# tf-idf ridge ensemble 1st & toxic comment
refer to following url  
https://www.kaggle.com/vitaleey/tfidf-ridge

In [ ]:
class Config:
    seed = 42
    cv_num = 3
    emb_flag = 0

In [ ]:
!nvidia-smi

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import numpy as np
import os
import re
import random
import gc
pd.set_option('display.max_columns', None)
np.seterr(divide='ignore', invalid='ignore')
gc.enable()

import pickle

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR
# NLP
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification, BertModel, BertTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import lightgbm as lgbm
from scipy.sparse import csr_matrix, hstack, vstack



import seaborn as sns
import matplotlib.pyplot as plt
import glob
%matplotlib inline
from itertools import product
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import roc_curve, confusion_matrix, log_loss,plot_confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, KFold
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics, preprocessing
from sklearn.decomposition import TruncatedSVD
import plotly.graph_objects as go
from collections import defaultdict
import unidecode
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
import unidecode

# Random Seed Initialize

def seed_everything(seed=Config.seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
INPUT = '../input/'
data_dir = '../input/jigsaw-toxic-severity-rating'
val =  pd.read_csv(os.path.join(data_dir, 'validation_data.csv'))
train =  pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
comments_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
ruddit = pd.read_csv('../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv')

In [ ]:
# get from moritas code. thx
train_1st_test_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv')
train_1st_test_lb_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv')

#testデータとlabelくっつける
train_1st_test_df = train_1st_test_df.merge(train_1st_test_lb_df, on='id')

# -1を落とす
drop_idx = (train_1st_test_df.loc[:,"toxic":]==-1).sum(axis = 1) >=1
train_1st_test_df = train_1st_test_df[~drop_idx].reset_index(drop=True)

# trainとtestをくっつける
train = pd.concat([train,train_1st_test_df]).reset_index(drop = True)

In [ ]:
def save_pkl(file_name, processor):
    OUTPUT_DIR = './'
    file_name = os.path.join(OUTPUT_DIR,file_name)
    pickle.dump(processor,open(file_name, 'wb'))
    print("FINISH")
def load_pkl(file_path):
    out_object = pickle.load(open(file_path, 'rb'))   
    return out_object

In [ ]:
val.head()

In [ ]:
ruddit.head()

In [ ]:
train.head()

# EDA

In [ ]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
#     text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [ ]:
# extract unique txt
all_txt = pd.DataFrame({'text':pd.concat([val['less_toxic'], val['more_toxic']])}).drop_duplicates().reset_index(drop=True)
print('unique txt num:', len(all_txt))

In [ ]:
all_txt['dup_flag'] = 1
# all_txt = all_txt.rename(columns={'text':'comment_text'})

In [ ]:
train = train.merge(all_txt.rename(columns={'text':'comment_text'}), on='comment_text', how='left')
train = train.query('dup_flag != 1').reset_index(drop=True)

In [ ]:
# max_sen_length = 0
# tokenizer = AutoTokenizer.from_pretrained(Config.model)

# for sentence in tqdm(all_txt['text']):
#     token_words = tokenizer.encode_plus(sentence)['input_ids']
#     if max_sen_length < len(token_words):
#         max_sen_length = len(token_words)

# print('max_length:', max_sen_length)

In [ ]:
tqdm.pandas()
all_txt['cleaning'] = all_txt['text'].progress_apply(text_cleaning)

In [ ]:
def get_comb(sequence, tfv_col, emb_flag=False):
    if emb_flag  == 0:
        return tfv_col

# TFIDF

In [ ]:
train.head()

In [ ]:
# df['severe_toxic'] = df.severe_toxic * 2
# df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) ).astype(int)
# df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

# Create a score that measure how much toxic is a comment
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    train[category] = train[category] * cat_mtpl[category]

train['score'] = train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

train['y'] = train['score']

In [ ]:
train = train[['comment_text', 'y']].rename(columns={'comment_text':'text'})
train.head()

In [ ]:
train['y'] = (train['y'] - train['y'].min()) / (train['y'].max() - train['y'].min())
train['y'].hist()

In [ ]:
ruddit = ruddit[['txt', 'offensiveness_score']].rename(columns={'txt': 'text',
                                                                'offensiveness_score':'y'})

ruddit['y'] = (ruddit['y'] - ruddit['y'].min()) / (ruddit['y'].max() - ruddit['y'].min())
ruddit['y'].hist()

## undersampling

In [ ]:
train['y'].value_counts(normalize=True)

In [ ]:
# train = pd.concat([train[train.y>0] , 
#                 train[train.y==0].sample(int(len(train[train.y>0])*6.5)) ], axis=0).sample(frac=1)
train['y'].value_counts(normalize=True)

## preprocessing

In [ ]:
tqdm.pandas()
train['cleaning'] = train['text'].progress_apply(text_cleaning)

val['cleaning_less'] = val['less_toxic'].progress_apply(text_cleaning)
val['cleaning_more'] = val['more_toxic'].progress_apply(text_cleaning)

comments_df["cleaning"] = comments_df["text"].progress_apply(text_cleaning)

ruddit['cleaning'] = ruddit['text'].progress_apply(text_cleaning)

In [ ]:
tfv = TfidfVectorizer(min_df=3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5))
tfv_col = tfv.fit_transform(train['cleaning'])
# save_pkl('kazuma_tfv_toxic_comment', tfv)

tfv_rud = TfidfVectorizer(min_df=3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5))
tfv_col_rud = tfv_rud.fit_transform(ruddit['cleaning'])
# save_pkl('kazuma_tfv_ruddit', tfv)

In [ ]:
comb = get_comb(train['cleaning'], tfv_col, Config.emb_flag)

comb_rud = get_comb(ruddit['cleaning'], tfv_col_rud, Config.emb_flag)

# create dataset

In [ ]:
train_x = comb
del comb
train_y = train['y']
gc.collect()

ruddit_x = comb_rud
del comb_rud
ruddit_y = ruddit['y']
gc.collect()

## model

In [ ]:
def create_multi_models(alpha_list, x, y):
    models = []
    for v in alpha_list:
        model = Ridge(alpha=v)
        model.fit(x, y)
        models.append(model)
    return models

In [ ]:
alpha_list = [0.5, 1, 2]
train_models = create_multi_models(alpha_list, train_x, train_y)
ruddit_models = create_multi_models(alpha_list, ruddit_x, ruddit_y)

## preprocessor and models saving

In [ ]:
processor_dict = {'toxc_comment_preprocessor': tfv,
                  'ruddit_preprocessor': tfv_rud}

save_pkl('kazuma_processor.pkl', processor_dict)
save_pkl('kazuma_models_toxic.pkl', train_models)
save_pkl('kazuma_models_ruddit.pkl', ruddit_models)

## validation

In [ ]:
X_less = tfv.transform(val['cleaning_less'])
X_more = tfv.transform(val['cleaning_more'])
comb_less = get_comb(val['cleaning_less'], X_less, Config.emb_flag)
comb_more = get_comb(val['cleaning_more'], X_more, Config.emb_flag)
val_x_more = comb_more
val_x_less = comb_less

X_less_rud = tfv_rud.transform(val['cleaning_less'])
X_more_rud = tfv_rud.transform(val['cleaning_more'])
comb_less_rud = get_comb(val['cleaning_less'], X_less_rud, Config.emb_flag)
comb_more_rud = get_comb(val['cleaning_more'], X_more_rud, Config.emb_flag)
val_x_more_rud = comb_more_rud
val_x_less_rud = comb_less_rud

del comb_more
del X_more
del comb_less
del X_less
gc.collect()

In [ ]:
def ensemble(data, preds, cv_num):
    if type(data) == type(csr_matrix(1)):
        pred = np.zeros((data.shape[0],))
    else:
        pred = np.zeros((len(data),))        

    for v in preds:
      pred += v

    return pred/cv_num

def multi_predict(test_x, models):
    preds = []
    for model in models:
        pred = model.predict(test_x)
        preds.append(pred)
    return preds

In [ ]:
preds_less = multi_predict(val_x_less, train_models)
preds_more = multi_predict(val_x_more, train_models)

In [ ]:
preds_less_rud = multi_predict(val_x_less_rud, ruddit_models)
preds_more_rud = multi_predict(val_x_more_rud, ruddit_models)

In [ ]:
preds_less_ens = ensemble(val_x_less, preds_less, 3)
preds_more_ens = ensemble(val_x_more, preds_more, 3)
preds_less_rud_ens = ensemble(val_x_less_rud, preds_less_rud, 3)
preds_more_rud_ens = ensemble(val_x_more_rud, preds_more_rud, 3)

In [ ]:
val['preds_less'] = preds_less_ens
val['preds_more'] = preds_more_ens
val['preds_less_rud'] = preds_less_rud_ens
val['preds_more_rud'] = preds_more_rud_ens

In [ ]:
print((val['preds_less'] < val['preds_more']).mean())
print((val['preds_less_rud'] < val['preds_more_rud']).mean())

In [ ]:
val.to_csv('kazuma_val.csv', index=False)